## Baseline-решение предсказания свойств флуоресцентных белков

### Подготовка к работе

In [ ]:
! pip install scikit-learn pandas numpy 

In [153]:
from ast import literal_eval
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

### Загрузка данных

Загрузим данные из csv файла. Разделеним на обучающую и тестовую выборки (80/20).

In [ ]:
df = pd.read_csv("dataset.csv")
df_train, df_test = train_test_split(df, test_size=0.2, random_state=2009) # test_size - размер тестовой выборки

### KNN

Извлечем последовательности и целевые значения из датафрейма. Пишем только те у которых есть числовые данные в таргетах.

In [ ]:
def get_target(df, target):
    sequences_list = []
    target_list = []

    for idx, row in df.iterrows():
        if pd.notna(row[target]):
            if isinstance(row['sequence'], str):
                try:
                    seq = literal_eval(row['sequence'])
                except (SyntaxError, ValueError):
                    seq = np.fromstring(
                        row['sequence'].strip('[]'), 
                        sep=' ', 
                        dtype=np.float32
                    )
            else:
                seq = row['sequence']
            
            sequences_list.append(seq)
            target_list.append(row[target])

    return [np.array(sequences_list), np.array(target_list)] # [эмбеддинги, свойства]

Настроим KNN регрессию, найдем лучшее количества соседей (k) методом кросс-валидации.

In [ ]:
def regressor_knn(X_train, X_test, y_train, y_test):
    params = {'n_neighbors' : range(1, 30)}

    knn_regressor = KNeighborsRegressor()
    grid = GridSearchCV(knn_regressor, params, cv=5, scoring='neg_mean_squared_error') # MSE
    grid.fit(X_train, y_train)

    knn_regressor = KNeighborsRegressor(n_neighbors=grid.best_params_['n_neighbors'])
    knn_regressor.fit(X_train, y_train)
    predictions = knn_regressor.predict(X_test)

    return [np.sqrt(-grid.best_score_), grid.best_params_['n_neighbors'], knn_regressor, predictions] # метрики, гиперпараметры и предсказанные значения

Настроим KNN классификацию, найдем лучшее количество соседей (k) методом кросс-валидации.

In [ ]:
def classification_knn(X_train, X_test, y_train, y_test):
    params = {'n_neighbors' : range(1, 30)} # ищем k ближайщих соседей из диапазона 

    knn_classif = KNeighborsClassifier()
    grid = GridSearchCV(knn_classif, params, cv=5, scoring='accuracy')
    grid.fit(X_train, y_train)

    scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted'] # метрики
    results = cross_validate(knn_classif, X_train, y_train, cv=5, scoring=scoring)

    knn_classif = KNeighborsClassifier(n_neighbors=grid.best_params_['n_neighbors'])
    knn_classif.fit(X_train, y_train)
    predictions = knn_classif.predict(X_test)

    return [results, grid.best_score_, grid.best_params_['n_neighbors'], knn_classif, predictions] # метрики, гиперпараметры и предсказанные значения

### Результаты и метрики

In [ ]:
def metrics(results, target):
    if len(results) == 4:
        print(f'{target:<12} | RMSE: {results[0]}, k: {results[1]}')
    else:
        print(
            f'{target:<12} | accuracy: {results[0]['test_accuracy'].mean()}' + 
            f', precision: {results[0]['test_precision_weighted'].mean()}' + 
            f', recall: {results[0]['test_recall_weighted'].mean()}' +
            f', f1: {results[0]['test_f1_weighted'].mean()}'
        )

Выведем полученные метрики.

In [ ]:
target = ['brightness', 'em_max', 'ex_max', 'ext_coeff', 'lifetime', 'maturation', 'pka', 'stokes_shift', 'qy', 'agg'] # свойства белка

for item in target:
    params_train = get_target(df_train, item)
    params_test = get_target(df_test, item)

    if item != 'agg':
        metrics(regressor_knn(params_train[0], params_test[0], params_train[1], params_test[1]), item)
    else:
        metrics(classification_knn(params_train[0], params_test[0], params_train[1], params_test[1]), item)